In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import json
from gensim.utils import deaccent

unable to import 'smart_open.gcs', disabling that module


In [67]:
class Downloader():
    def __init__(self):
        self.link = 'https://www.bezrealitky.cz/vypis/nabidka-pronajem/byt/praha?_token=pr1lf-vKwDFfmFbICiz2PfC-Zdwq-2JolXi4MeMHsrw&page=1'
        self.request = requests.get(self.link)
        self.request.encoding='UTF-8'
        self.soup = BeautifulSoup(self.request.text,'lxml')
        
        self.page_bezrealitky = int(self.soup.findAll('a',{'class':'page-link pagination__page'})[-2].text)
        self.hrefs_bezrealitky = ['https://www.bezrealitky.cz/vypis/nabidka-pronajem/byt/praha?_token=pr1lf-vKwDFfmFbICiz2PfC-Zdwq-2JolXi4MeMHsrw&page=' 
                                  + str(i) for i in range(1,self.page_bezrealitky)]
        self.soups = []
        
    def get_soups(self):
        for link in self.hrefs_bezrealitky:
            sleep(1)
            self.link = link
            self.request = requests.get(self.link)
            self.request.encoding='UTF-8'
            self.soups.append(BeautifulSoup(self.request.text,'lxml'))
        with open('bezrealitky_links.txt', 'w') as f:
            for s in self.soups:
                f.write(str(deaccent(s).encode("utf-8")) + 'BREAKHERE')
            f.close
        
    def get_data(self):
        with open('bezrealitky_links.txt', 'r') as f:
            content = f.read()
        soup_list = content.split('BREAKHERE')
        dicts = {}
        counter = 0
        for soup in soup_list:
            descrips = []
            values = []
            vals = BeautifulSoup(soup,'lxml').findAll('strong', {'class':'product__value'})
            ##vals = soup.findAll('strong', {'class':'product__value'})
            for vl in vals:
                values.append(vl.text.strip())
            #img = soup.findAll('img')
            img = BeautifulSoup(soup,'lxml').findAll('img')
            for i in img:
                if 'Pronajem' and 'obr. c. 1' in i['alt']:
                        info = i['alt'].split(',')[0:4]
                        if 'Praha' == info[-1].strip():
                            info.insert(2, 'NaN')
                            del info[-1]
                            m = info[1].split(' ')
                            info[1] = m[1]
                            descrips.append(info)
                        else:
                            m = info[1].split(' ')
                            info[1] = m[1]
                            descrips.append(info)
            count = 0
            for pp in values:
                try:
                    descrips[count].append(pp)
                    descrips[count][0] = descrips[count][0][-4:].strip()
                    count += 1
                except IndexError:
                    count += 1
                    continue
            for item in descrips:
                try:
                    if '+' in item[4]:
                        prices = item.pop(4).split('+')
                        item.append(re.sub("[^0-9]", "", prices[0]))
                        item.append(re.sub("[^0-9]", "", prices[1]))
                    else:
                        prices = [item.pop(4), '0']
                        item.append(re.sub("[^0-9]", "", prices[0]))
                        item.append(re.sub("[^0-9]", "", prices[1]))
                except IndexError:
                    continue
            for item in descrips:
                try:
                    dict = {}
                    dict['Size'] = item[0]
                    dict['m2'] = re.sub("[^0-9]", "", item[1])
                    dict['Street'] = deaccent(item[2])
                    dict['District'] = deaccent(item[3])
                    dict['Base Price'] = int(item[4])
                    dict['Utilities Price'] = int(item[5])
                    dict['Total Price'] = int(item[4]) + int(item[5])
                    dicts[counter] = dict
                    counter += 1
                except IndexError:
                    counter +=1
                    continue
            print('Done loop number ' + str(counter) + '. Printing descrips.')
            print(descrips)
        with open('bezrealitky.json', 'w') as write_file:
            json.dump(dicts, write_file, indent = 4)

        
            
        

In [69]:
a = Downloader()

In [65]:
a.get_soups()

In [70]:
a.get_data()

Done loop number 10. Printing descrips.
[['3+kk', '66', ' Hlavackova', ' Praha - Kosire', '19900', '3900'], ['1+kk', '31', ' Hlavackova', ' Praha - Kosire', '11990', '1900'], ['2+kk', '47', ' Chrudimska', ' Praha - Vinohrady', '13000', '2500'], ['2+kk', '54', ' Zelenecska', ' Praha - Hloubetin', '15700', '2200'], ['2+kk', '86', ' Konevova', ' Praha - Zizkov', '25990', '3900'], ['2+kk', '60', ' Roztocka', ' Praha - Sedlec', '15000', '2000'], ['2+1', '62', ' Naprstkova', ' Praha - Stare Mesto', '14000', '1000'], ['2+1', '60', ' Vysehradska', ' Praha - Nove Mesto', '24500', '3000'], ['1+1', '31', ' V Haji', ' Praha - Holesovice', '10000', '2475'], ['3+1', '68', ' Na Krivine', ' Praha - Michle', '14500', '5500']]
Done loop number 20. Printing descrips.
[['3+kk', '74', ' Sekaninova', ' Praha - Nusle', '20500', '1500'], ['3+kk', '82', ' Na Obratce', ' Praha - Hloubetin', '16000', '3000'], ['3+kk', '72', ' 5. kvetna', ' Praha - Nusle', '14000', '4500'], ['2+kk', '71', ' Navratilova', ' Praha 

[['2+kk', '48', ' U libenskeho pivovaru', ' Praha - Liben', '11300', '4700'], ['2+kk', '59', ' Na Palouku', ' Praha - Strasnice', '14500', '3000'], ['2+kk', '52', ' Baranova', ' Praha - Zizkov', '19000', '4000'], ['1+kk', '30', ' Ke Dzbanu', ' Praha - Liboc', '14000', '2000'], ['4+1', '99', ' Na Strelnici', ' Praha - Karlin', '22000', '3000'], ['1+kk', '20', ' Na Cikance', ' Praha - Radotin', '4000', '2000'], ['3+kk', '68', ' Nevanova', ' Praha - Repy', '17900', '3000'], ['3+kk', '84', ' Rohacova', ' Praha - Zizkov', '19000', '4000'], ['2+1', '60', ' Wuchterlova', ' Praha - Dejvice', '15990', '3000'], ['2+kk', '44', ' Sarajevska', ' Praha - Vinohrady', '13000', '1000']]
Done loop number 130. Printing descrips.
[['2+kk', '45', ' Libicka', ' Praha - Vinohrady', '13000', '3500'], ['3+kk', '96', ' Ke Klimentce', ' Praha - Smichov', '32000', '3500'], ['3+1', '110', ' Varsavska', ' Praha - Vinohrady', '29000', '4000'], ['2+kk', '60', ' Taskentska', ' Praha - Vrsovice', '12000', '4000'], ['3+

[['2+kk', '44', ' Nad Kajetankou', ' Praha - Brevnov', '14800', '1200'], ['4+1', '88', ' Podjavorinske', ' Praha - Chodov', '20900', '3500'], ['2+1', '45', ' Karanska', ' Praha - Malesice', '12000', '2700'], ['2+kk', '76', ' Vaclavske namesti', ' Praha - Nove Mesto', '16000', '2500'], ['2+kk', '57', 'NaN', ' Praha - Karlin', '18000', '3500'], ['2+kk', '54', ' Haluv Statek', ' Praha - Hostivar', '15000', '3500'], ['3+kk', '78', ' Na Brevnovske plani', ' Praha - Brevnov', '18500', '3500'], ['2+kk', '51', ' Zateckych', ' Praha - Nusle', '12500', '2000'], ['1+1', '35', 'NaN', ' Praha - Zabehlice', '9500', '2000'], ['3+1', '101', ' Benediktska', ' Praha - Stare Mesto', '30000', '1000']]
Done loop number 240. Printing descrips.
[['2+kk', '45', ' Dittrichova', ' Praha - Nove Mesto', '14500', '1760'], ['2+kk', '54', ' Manesova', ' Praha - Vinohrady', '18000', '1000'], ['3+1', '101', ' Benediktska', ' Praha - Stare Mesto', '30000', '1000'], ['3+1', '93', ' Stepanska', ' Praha - Nove Mesto', '19

[['1+1', '46', ' Petrohradska', ' Praha - Vrsovice', '14000', '2500'], ['3+kk', '70', ' Pratelstvi', ' Praha - Uhrineves', '17000', '4500'], ['3+kk', '55', ' Safarikova', ' Praha - Vinohrady', '14000', '4500'], ['2+1', '65', ' Benediktska', ' Praha - Stare Mesto', '14000', '2000'], ['2+kk', '40', ' Naprstkova', ' Praha - Stare Mesto', '12000', '2000'], ['3+1', '80', ' Naprstkova', ' Praha - Stare Mesto', '15000', '2000'], ['2+kk', '50', ' Zatecka', ' Praha - Josefov', '12000', '1000'], ['3+kk', '77', ' Hybernska', ' Praha - Nove Mesto', '18000', '2000'], ['3+kk', '48', ' Molakova', ' Praha - Karlin', '15950', '3800'], ['bytu', '17', ' Borivojova', ' Praha - Zizkov', '8000', '2000']]
Done loop number 349. Printing descrips.
[['1+1', '78', ' Nikolaje Alexejevice Nekrasova', ' Praha - Dejvice', '16000', '1900'], ['2+1', '55', ' Hybernska', ' Praha - Nove Mesto', '18000', '2000'], ['2+kk', '44', ' Obloukova', ' Praha - Vrsovice', '14000', '2500'], ['1+kk', '55', ' Hanusova', ' Praha - Mich

[['2+kk', '39', ' Dykova', ' Praha - Vinohrady', '11800', '2200'], ['2+kk', '43', ' Na Rokytce', ' Praha - Liben', '18000', '0'], ['2+kk', '63', ' Skrabkovych', ' Praha - Letnany', '15000', '2000'], ['2+kk', '50', ' Holeckova', ' Praha - Smichov', '14000', '800'], ['2+kk', '47', ' Pisecka', ' Praha - Vinohrady', '13500', '2500'], ['1+kk', '27', ' Vrsovicka', ' Praha - Vrsovice', '7000', '3500'], ['iera', '30', ' Platonova', ' Praha - Modrany', '9000', '1500'], ['bytu', '107', ' Uzbecka', ' Praha - Vrsovice', '19000', '4000'], ['5+1', '180', ' Farni', ' Praha - Stresovice', '65000', '10000'], ['iera', '17', ' Novakovych', ' Praha - Liben', '10900', '1000']]
Done loop number 458. Printing descrips.
[['bytu', '500', ' Moskevska', ' Praha - Vrsovice', '260000', '7500'], ['1+kk', '30', ' Jaurisova', ' Praha - Nusle', '11000', '700'], ['3+kk', '108', ' Laubova', ' Praha - Vinohrady', '49000', '5000'], ['2+1', '49', ' Novakovych', ' Praha - Liben', '16500', '1200'], ['iera', '18', ' Podolska'

...

[['1+kk', '27', ' Slezska', ' Praha - Vinohrady', '12000', '1500'], ['4+kk', '142', ' Premyslovska', ' Praha - Vinohrady', '39000', '4000'], ['2+kk', '69', ' Naskove', ' Praha - Kosire', '21500', '3500'], ['2+kk', '50', ' Rehorova', ' Praha - Zizkov', '14000', '1827'], ['3+kk', '156', ' Krizikova', ' Praha - Karlin', '42200', '4000'], ['2+kk', '79', ' Krizikova', ' Praha - Karlin', '21400', '4000'], ['2+kk', '45', ' Werichova', ' Praha - Hlubocepy', '12000', '3000'], ['1+kk', '40', ' Krizikova', ' Praha - Karlin', '14000', '4000'], ['1+kk', '30', ' Trojicka', ' Praha - Nove Mesto', '12000', '1500'], ['2+kk', '44', ' Rybalkova', ' Praha - Vinohrady', '20000', '4000']]
Done loop number 2315. Printing descrips.
[['iera', '21', ' Na Padesatem', ' Praha - Strasnice', '9500', '2500'], ['iera', '40', ' Nad Panenskou', ' Praha - Stresovice', '13000', '700'], ['3+1', '80', 'NaN', ' Praha - Kobylisy', '17000', '4000'], ['1+kk', '23', ' Malatova', ' Praha - Smichov', '8500', '2000'], ['1+kk', '42

[['1+1', '34', 'NaN', ' Praha - Smichov', '12000', '4500'], ['1+kk', '38', ' Makedonska', ' Praha - Strizkov', '13000', '2100'], ['2+kk', '33', ' V Krovinach', ' Praha - Branik', '12500', '2000'], ['2+kk', '61', ' Mrazkova', ' Praha - Cerny Most', '16000', '2212'], ['iera', '30', ' Cernochova', ' Praha - Kosire', '10500', '2500'], ['2+1', '63', ' Evropska', ' Praha - Vokovice', '22500', '0'], ['3+kk', '76', ' Ceskoslovenskeho Exilu', ' Praha - Modrany', '16000', '2500'], ['1+kk', '38', ' Olsanska', ' Praha - Zizkov', '24000', '3000'], ['1+kk', '35', ' Olsanska', ' Praha - Zizkov', '12800', '2850']]
Done loop number 2424. Printing descrips.
[['3+kk', '99', 'NaN', ' Praha - Liben', '21000', '5500'], ['3+1', '64', ' Jungmannovo Namesti', ' Praha - Nove Mesto', '17000', '3000'], ['2+kk', '53', ' Karla Englise', ' Praha - Smichov', '22000', '3000'], ['1+kk', '28', ' Sudomerska', ' Praha - Zizkov', '13900', '2900'], ['2+kk', '67', ' Harmonicka', ' Praha - Stodulky', '16500', '3000'], ['3+kk'

[['2+kk', '52', ' Hradebni', ' Praha - Stare Mesto', '22000', '2500'], ['3+1', '116', ' Za Postou', ' Praha - Strasnice', '23300', '4200'], ['3+kk', '95', ' Krocinova', ' Praha - Stare Mesto', '28000', '1000'], ['3+1', '80', ' Janackovo Nabrezi', ' Praha - Smichov', '22000', '3500'], ['2+kk', '54', ' Na Plzence', ' Praha - Smichov', '15000', '1000'], ['3+kk', '95', ' Rehorova', ' Praha - Zizkov', '19000', '4000'], ['2+kk', '45', ' Rumunska', ' Praha - Vinohrady', '16000', '6000'], ['2+kk', '60', ' Strakonicka', ' Praha - Smichov', '21000', '3500'], ['1+kk', '34', ' Honzikova', ' Praha - Dolni Mecholupy', '19000', '4000']]
Done loop number 2532. Printing descrips.
[['2+kk', '63', 'NaN', ' Praha - Vrsovice', '19000', '4000'], ['1+1', '50', ' Zenklova', ' Praha - Liben', '14000', '1'], ['2+kk', '54', ' Slezska', ' Praha - Vinohrady', '19000', '0'], ['3+kk', '75', ' Pod Strani', ' Praha - Strasnice', '24000', '2100'], ['3+kk', '70', ' Blodkova', ' Praha - Zizkov', '25900', '3900'], ['1+1',

[['2+kk', '52', ' Nezamyslova', ' Praha - Nusle', '22000', '3000'], ['1+kk', '26', ' Luka Living', ' Praha - Stodulky', '13900', '0'], ['2+kk', '62', ' Honzikova', ' Praha - Dolni Mecholupy', '18000', '2500'], ['2+kk', '41', ' Cajkovskeho', ' Praha - Zizkov', '15000', '1850'], ['1+kk', '18', ' Freyova', ' Praha - Vysocany', '12500', '1500'], ['1+1', '27', ' Francouzska', ' Praha - Vrsovice', '13000', '0'], ['2+1', '62', ' Novorossijska', ' Praha - Vrsovice', '8000', '500'], ['4+kk', '98', ' V Doline', ' Praha - Michle', '17000', '4379'], ['1+kk', '57', ' Pelhrimovska', ' Praha - Michle', '29900', '0']]
Done loop number 2641. Printing descrips.
[['2+kk', '50', ' Kamenicka', ' Praha - Holesovice', '18000', '4000'], ['iera', '35', ' Krakovska', ' Praha - Nove Mesto', '14000', '3000'], ['3+1', '97', ' U Stareho Hrbitova', ' Praha - Josefov', '25000', '4000'], ['2+1', '90', ' Soukenicka', ' Praha - Nove Mesto', '19900', '4000'], ['2+kk', '60', ' Farskeho', ' Praha - Holesovice', '15000', '3

Done loop number 2746. Printing descrips.
[['4+kk', '115', ' Netlucka', ' Praha - Dubec', '19000', '4500'], ['2+kk', '35', ' Nad Primaskou', ' Praha - Strasnice', '13000', '1700'], ['1+kk', '43', 'NaN', ' Praha - Zlicin', '11500', '3000'], ['2+kk', '50', ' Luka Living', ' Praha - Stodulky', '15500', '0'], ['1+kk', '27', ' Adamovska', ' Praha - Michle', '9500', '3000'], ['1+kk', '31', 'NaN', ' Praha - Vysocany', '13900', '2500'], ['2+1', '74', ' Nad Rokoskou', ' Praha - Liben', '17000', '4000'], ['3+1', '72', ' Malkovskeho', ' Praha - Letnany', '20000', '3000'], ['1+kk', '41', ' Petyrkova', ' Praha - Chodov', '10000', '3500'], ['2+1', '45', ' Maroldova', ' Praha - Nusle', '17000', '2000']]
Done loop number 2756. Printing descrips.
[['1+1', '35', 'NaN', ' Praha - Brevnov', '14000', '500'], ['2+kk', '53', ' Druzstevni', ' Praha - Nusle', '13900', '2600'], ['2+kk', '57', ' Rohanske nabrezi', ' Praha - Karlin', '23000', '5500'], ['2+kk', '80', ' Musilkova', ' Praha - Kosire', '21500', '2500